In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

# AWS helper

Resourses and useful links:

 - ...

In [2]:
#|default_exp aws
#|export
from nbx.utils import run_bash, Bunch
from nbx.fileio import load, dump
from nbx.cli import add_argparse
import os
from pathlib import Path


In [3]:
os.getcwd()

'/Users/mirko/Workspace/nbx/notebooks'

In [4]:
#|export
def _test(public_IP, *, user="root"):
    """Just for testing."""
    print(os.getcwd())
    
test = add_argparse(_test)

In [5]:
#|export
def _aws_init(public_IP, *, user="root", key):
    """."""
    fname = Path("~/.nbx/aws.json")
    conf = Bunch()
    dump(conf, fname)
    
aws_init = add_argparse(_aws_init)

In [6]:
#|export
import subprocess
import sys    

SSH_CONFIG_TEMPLATE = """
Host {host}
    User {user}
    HostName {ip}
    IdentityFile {key}
    
"""

def _ssh(ip, *, user=None, key=None, port=8888):
    """ssh-connect to ip address, with an optional username (default is 'ubuntu')."""
    # Need to replace ".", because I don't want 
    # `ip` to be a keyword argument
    if ip == "." or ip == "...": ip = None    
    func_args = locals()
    conf = Bunch(**func_args)
        
    # Loads default config if there is one
    # and update the conf object with data
    # from it, but function args have precedence
    fname = os.path.expanduser("~/.nbx/aws.json")
    fname = Path(fname)
    if fname.is_file(): 
        stored = load(fname)
        for k,v in stored.items():
            # Function args have precedence
            if conf[k] is None: conf[k] = v
        
    # Check if we got everything we need to
    # connect to instance
    fail = False
    for k in ["ip", "user", "key", "port"]:
        if conf[k] is None:
            fail = True
            print(f"Please provide --{k}")
            
    # Save what we already got, and
    # proceed if we got everything or return
    dump(conf, fname)
    if fail: return
    
    config_str  = SSH_CONFIG_TEMPLATE.format(
        host="aws", 
        user=conf.user, 
        ip=conf.ip, 
        key=conf.key
    )
    print(config_str)
    dump(config_str, os.path.expanduser("~/.ssh/ec2_config"), format=".txt")
    
    # We could write some environment vars
    # but we can't source them from here
    #
    #     fname = os.path.expanduser("~/.nbx/.bash_aws")
    #     string  = f"export xaws={conf.user}@{conf.ip};\n"
    #     dump(string, fname, format=".txt")

    # Connect to server and forward local port 8888 to remote port 8888
    # We can now connect to a remote jupyter notebook server via `http://localhost:8888/`
    cmd = f"ssh -i {conf.key} -L {conf.port}:localhost:{conf.port} {conf.user}@{conf.ip}"
    os.system(f'bash -c \"{cmd}\"')
    
    


ssh = add_argparse(_ssh)

In [ ]:
#|export
import subprocess
import sys    

def _scp(sources, target, ip=None, user=None, key=None):
    """scp..."""
    conf = Bunch(ip=ip,user=user,key=key)
                
    # Loads default config if there is one
    # and update the conf object with data
    # from it
    fname = os.path.expanduser("~/.nbx/aws.json")
    fname = Path(fname)
    if fname.is_file(): 
        stored = load(fname)
        for k,v in stored.items():
            if v is not None: conf[k] = v
        
    # Check if we got everything we need to
    # connect to instance
    fail = False
    for k in ["ip", "user", "key"]:
        if conf[k] is None:
            fail = True
            print(f"Please provide --{k}")
            
    # Save what we already got, and
    # proceed if we got everything or return
    dump(conf, fname)
    if fail: return
    
    
    
    target = target.replace("_rem_",  f"{conf.user}@{conf.ip}")    
    target = target.replace("_home_", f"{conf.user}@{conf.ip}:~")    
    ops = ""
    if "_rem_" not in sources[0] and Path(sources[0]).is_dir():
        ops = "-r"
    for i in range(len(sources)): 
        sources[i] = sources[i].replace("_rem_",  f"{conf.user}@{conf.ip}")
        sources[i] = sources[i].replace("_home_", f"{conf.user}@{conf.ip}:~")
        
    print(f"Copying")
    for source in sources: print(f"\t{source}")
    print(f"to\n\t{target}")

    sources = " ".join(sources)
    


    # Connect to server and forward local port 8888 to remote port 8888
    # We can now connect to a remote jupyter notebook server via `http://localhost:8888/`
    cmd = f"scp -i {conf.key} {ops} {sources} {target}"

    os.system(f'bash -c \"{cmd}\"')


def scp():
    sources = sys.argv[1:-1]
    target = sys.argv[-1]
    _scp(sources, target)


In [11]:
Path("nbx_example").is_dir()

True

In [13]:
#|export

def _tunnel(port):
    """ssh and tunnel port..."""
    func_args = locals()
    conf = Bunch(**func_args)
        
    # Loads default config if there is one
    # and update the conf object with data
    # from it, but function args have precedence
    fname = os.path.expanduser("~/.nbx/aws.json")
    fname = Path(fname)
    if fname.is_file(): 
        stored = load(fname)
        for k,v in stored.items():
            if k not in conf: conf[k] = v
    
    # Check if we got everything we need to
    # connect to instance
    fail = False
    for k in ["ip", "user", "key", "port"]:
        if conf[k] is None:
            fail = True
            print(f"Please provide --{k}")
            
    if fail: return
    
    # We could write some environment vars
    # but we can't source them from here
    #
    #     fname = os.path.expanduser("~/.nbx/.bash_aws")
    #     string  = f"export xaws={conf.user}@{conf.ip};\n"
    #     dump(string, fname, format=".txt")

    # Connect to server and forward local port 8888 to remote port 8888
    # We can now connect to a remote jupyter notebook server via `http://localhost:8888/`
    cmd = f"ssh -i {conf.key} -L {conf.port}:localhost:{conf.port} {conf.user}@{conf.ip}"
    os.system(f'bash -c \"{cmd}\"')


tunnel = add_argparse(_tunnel)